# Split Apply Combine

I first heard about this concept in [this HN comment](https://news.ycombinator.com/item?id=18354369). It links to [this panadas doc](https://pandas.pydata.org/pandas-docs/stable/groupby.html) from which this nb will draw. It also mentions [this doc](https://vita.had.co.nz/papers/plyr.pdf) for an R library that I don't fully follow.

I'll use [these wine reviews](https://www.kaggle.com/zynicide/wine-reviews) as a sample dataset.

See also:
* [API reference for GroupBy Objects](https://pandas.pydata.org/pandas-docs/stable/api.html#groupby)
* [Pandas GroupBy object](http://pandas.pydata.org/pandas-docs/stable/groupby.html#groupby-object-attributes) (part of that first doc but a bit hard to find)
* [Dispatching](http://pandas.pydata.org/pandas-docs/stable/groupby.html#groupby-dispatch), or how pandas knows what you mean when you say `"sum"`


Summary:
I'm not sure that this specific workflow is better for me in most cases. It doesn't seem to handle floats in the most natural way (hard to group by float, you need an intermediate mapping of float to group) which is a lot of what I do. There also seems to be quite a bit of work to do on this functionality in pandas (see the linked github issues).
However, I don't think I lose anything by using pandas dataframes rather than numpy structured arrays by default (if anything the column major order is more useful for me). And if that allows occasional workflows like this it is good to know about.

The biggest win of going through this is starting to think about problems like this. Whether we are doing a `dataframe.groupby` or a `scipy.binned_statistic` a lot of analysis comes down to:
1. Bin the data
2. Do something to data within that bin (maybe a transform - e.g. from a delta to an effect size, maybe a aggregation)
3. Plots, print, etc the result of that transform/aggregation

## Imports etc

In [32]:
import numpy as np
import pandas as pd
import scipy.stats

%run ~/.jupyter/config.ipy

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [84]:
data = pd.read_csv("data/winemag-data-130k-v2.csv", index_col=0)
print(list(data))
data.head()

['country', 'description', 'designation', 'points', 'price', 'province', 'region_1', 'region_2', 'taster_name', 'taster_twitter_handle', 'title', 'variety', 'winery']


,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


# Detailed example

In [83]:
# Which country has the best wine?
display(data.groupby("country")["points"].mean().sort_values(ascending=False).head())

country
England    91.581081
India      90.222222
Austria    90.101345
Germany    89.851732
Canada     89.369650
Name: points, dtype: float64

## Split (group by)

In [4]:
# Ok, what is going on here
gb = data.groupby("country")
print(type(gb))

print("\nInternally this is roughly just a dict with the keys being the groups and the values the indexes")
print(gb.groups.keys())
print(gb.groups["England"]) # or 
assert len(gb) == len(list(set(list(data["country"])))) - 1 # groupby excludes nans!

print("\nWe can also iterate through the groups")
for name, group in gb:
    print(name)
    break
display(group.head()) # group here is just a normal dataframe (I'm pretty sure just a view into the original df)

print("\nOr we can get the group (the view into the DF, not just the indexes) like so:")
display(gb.get_group("England").head())

<class 'pandas.core.groupby.groupby.DataFrameGroupBy'>

Internally this is roughly just a dict with the keys being the groups and the values the indexes
dict_keys(['Argentina', 'Armenia', 'Australia', 'Austria', 'Bosnia and Herzegovina', 'Brazil', 'Bulgaria', 'Canada', 'Chile', 'China', 'Croatia', 'Cyprus', 'Czech Republic', 'Egypt', 'England', 'France', 'Georgia', 'Germany', 'Greece', 'Hungary', 'India', 'Israel', 'Italy', 'Lebanon', 'Luxembourg', 'Macedonia', 'Mexico', 'Moldova', 'Morocco', 'New Zealand', 'Peru', 'Portugal', 'Romania', 'Serbia', 'Slovakia', 'Slovenia', 'South Africa', 'Spain', 'Switzerland', 'Turkey', 'US', 'Ukraine', 'Uruguay'])
Int64Index([  4073,   6700,   9909,  22156,  22369,  22540,  24839,  27356,
             27364,  27848,  27872,  31166,  41914,  45533,  45590,  47458,
             47460,  47731,  48119,  53623,  53629,  54131,  54133,  54686,
             56417,  56611,  56660,  58496,  58747,  58752,  59857,  59863,
             60002,  60027,  62352,  62

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
16,Argentina,"Baked plum, molasses, balsamic vinegar and che...",Felix,87,30.0,Other,Cafayate,NaN,Michael Schachner,@wineschach,Felix Lavaque 2010 Felix Malbec (Cafayate),Malbec,Felix Lavaque
17,Argentina,Raw black-cherry aromas are direct and simple ...,Winemaker Selection,87,13.0,Mendoza Province,Mendoza,NaN,Michael Schachner,@wineschach,Gaucho Andino 2011 Winemaker Selection Malbec ...,Malbec,Gaucho Andino
183,Argentina,With attractive melon and other tropical aroma...,NaN,88,12.0,Other,Salta,NaN,Michael Schachner,@wineschach,Alamos 2007 Torrontés (Salta),Torrontés,Alamos
224,Argentina,Blackberry and road-tar aromas are dark and st...,Lunta,90,22.0,Mendoza Province,Luján de Cuyo,NaN,Michael Schachner,@wineschach,Mendel 2014 Lunta Malbec (Luján de Cuyo),Malbec,Mendel
231,Argentina,"Meaty and rubbery, but that's young Bonarda. T...",NaN,85,10.0,Mendoza Province,Mendoza,NaN,Michael Schachner,@wineschach,Andean Sky 2007 Bonarda (Mendoza),Bonarda,Andean Sky



Or we can get the group (the view into the DF, not just the indexes) like so:


,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
4073,England,A mix of fruity notes immediately hit: white c...,Blanc de Blancs,90,46.0,England,NaN,NaN,Anne Krebiehl MW,@AnneInVino,Ridgeview Estate 2011 Blanc de Blancs Chardonn...,Chardonnay,Ridgeview Estate
6700,England,Crisp green apple flavors are immediately appa...,Cuvée Brut,91,40.0,England,NaN,NaN,Anne Krebiehl MW,@AnneInVino,Wiston Estate Winery 2010 Cuvée Brut Sparkling...,Sparkling Blend,Wiston Estate Winery
9909,England,"Touches of vanilla, cream and lovely hints of ...",Blanc de Blancs,95,70.0,England,NaN,NaN,Anne Krebiehl MW,@AnneInVino,Nyetimber 2010 Blanc de Blancs Chardonnay (Eng...,Chardonnay,Nyetimber
22156,England,Golden color and hints of oatmeal on the nose ...,Three Graces,91,50.0,England,NaN,NaN,Anne Krebiehl MW,@AnneInVino,Chapel Down 2010 Three Graces Sparkling (England),Sparkling Blend,Chapel Down
22369,England,"Pure notes of red apple rise from the glass, p...",Rosé,94,65.0,England,NaN,NaN,Anne Krebiehl MW,@AnneInVino,Nyetimber NV Rosé Sparkling (England),Sparkling Blend,Nyetimber


## Apply

### Aggregation

Aggregation returns a data structure with fewer dimensions than the original data - e.g. mean: takes an array and returns an int.

In [79]:
gb = data.groupby("country")#, as_index=False) # With this added the country is no longer the index (just the first col)

mean_agg = gb.aggregate(np.mean) # .agg == .aggregate
print("Only columns where the function makes sense (numbers) are available in the aggregate:", list(mean_agg))
print("Note how the country is now the index")
print(type(mean_agg))
display(mean_agg.head())

print("Rather than pass a function (which accepts a list of values and returns a single value) there are also methods on the df:")
mean_agg = gb.mean()
display(mean_agg.head())

print("And we can get pretty much everything with describe:")
display(gb.describe().head())

print("We can do this manually with:")
my_agg = gb["points"].agg([np.sum, np.mean, np.std])
display(my_agg.head())

print("And we can get fully custom:")
custom_agg = gb.agg({
    "points": ["sum", np.mean],
    "price": [np.max, np.min, lambda price: np.max(price) - np.min(price)],
}).rename(columns={"<lambda>": "range"}) # this is pretty broken too https://github.com/pandas-dev/pandas/issues/18366
    
display(custom_agg.head())

Only columns where the function makes sense (numbers) are available in the aggregate: ['points', 'price']
Note how the country is now the index
<class 'pandas.core.frame.DataFrame'>


,points,price
country,,
Argentina,86.710263,24.510117
Armenia,87.500000,14.500000
Australia,88.580507,35.437663
Austria,90.101345,30.762772
Bosnia and Herzegovina,86.500000,12.500000


Rather than pass a function (which accepts a list of values and returns a single value) there are also methods on the df:


,points,price
country,,
Argentina,86.710263,24.510117
Armenia,87.500000,14.500000
Australia,88.580507,35.437663
Austria,90.101345,30.762772
Bosnia and Herzegovina,86.500000,12.500000


And we can get pretty much everything with describe:


points                                                 \
                         count       mean       std   min    25%   50%    75%   
country                                                                         
Argentina               3800.0  86.710263  3.179627  80.0  84.00  87.0  89.00   
Armenia                    2.0  87.500000  0.707107  87.0  87.25  87.5  87.75   
Australia               2329.0  88.580507  2.989900  80.0  87.00  89.0  91.00   
Austria                 3345.0  90.101345  2.499799  82.0  88.00  90.0  92.00   
Bosnia and Herzegovina     2.0  86.500000  2.121320  85.0  85.75  86.5  87.25   

                                price                                     \
                          max   count       mean        std   min    25%   
country                                                                    
Argentina                97.0  3756.0  24.510117  23.430122   4.0  12.00   
Armenia                  88.0     2.0  14.500000   0.707107  14.0  14.25   
Australia               100.0  2294.0  35.437663  49.049458   5.0  15.00   
Austria                  98.0  2799.0  30.762772  27.224797   7.0  18.00   
Bosnia and Herzegovina   88.0     2.0  12.500000   0.707107  12.0  12.25   

                                             
                         50%    75%     max  
country                                      
Argentina               17.0  25.00   230.0  
Armenia                 14.5  14.75    15.0  
Australia               21.0  38.00   850.0  
Austria                 25.0  36.50  1100.0  
Bosnia and Herzegovina  12.5  12.75    13.0

We can do this manually with:


,sum,mean,std
country,,,
Argentina,329499,86.710263,3.179627
Armenia,175,87.500000,0.707107
Australia,206304,88.580507,2.989900
Austria,301389,90.101345,2.499799
Bosnia and Herzegovina,173,86.500000,2.121320


And we can get fully custom:


points              price              
                           sum       mean    amax  amin   range
country                                                        
Argentina               329499  86.710263   230.0   4.0   226.0
Armenia                    175  87.500000    15.0  14.0     1.0
Australia               206304  88.580507   850.0   5.0   845.0
Austria                 301389  90.101345  1100.0   7.0  1093.0
Bosnia and Herzegovina     173  86.500000    13.0  12.0     1.0

### Transform

Returns something of the same size as the input, just transformed somehow.

Why do we need to group by first? Maybe we want to rank within the groups (e.g find the most expensive wine from each county). This looks like group by country then rank (transform price in dollars to rank).

Or maybe we want to fill NANs with some reasonable value for that group!

Though it actually looks like this is pretty broken/not feature complete. See https://github.com/pandas-dev/pandas/issues/17309

In [78]:
gb = data.groupby("country")["price"]

f = lambda x: x.fillna(x.mean())
# Note that this returns a dataframe/seris, not a group by. So we are now ungrouped
transformed = gb.transform(np.mean)
display(transformed.head())

0    39.663770
1    26.218256
2    36.573464
3    36.573464
4    36.573464
Name: price, dtype: float64

## Combine

Has kinda already been done for us - the apply usually returns a well defined object!

# Various examples

In [98]:
# Find the top 5 wines in each 10 dollar price bracket
data["price_bracket"] = data["price"] // 10

gb = data.groupby("price_bracket")

for price_backet, group in gb:
    print(price_backet)
    display(group.sort_values("points", ascending=False).head(5))
    print("etc...")
    break

0.0


,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,price_bracket
34629,Portugal,"Richly tannic, this concentrated wine has grea...",Toutalga,91,7.0,Alentejano,NaN,NaN,Roger Voss,@vossroger,Herdade dos Machados 2012 Toutalga Red (Alente...,Portuguese Red,Herdade dos Machados,0.0
43977,US,Kudos to Ste. Michelle for accurately labeling...,Dry,91,9.0,Washington,Columbia Valley (WA),Columbia Valley,Paul Gregutt,@paulgwine,Chateau Ste. Michelle 2010 Dry Riesling (Colum...,Riesling,Chateau Ste. Michelle,0.0
56988,US,"This tangy, all-stainless, lightly grassy, dry...",Fumé Blanc,91,9.0,Washington,Columbia Valley (WA),Columbia Valley,Paul Gregutt,@paulgwine,Barnard Griffin 2012 Fumé Blanc Sauvignon Blan...,Sauvignon Blanc,Barnard Griffin,0.0
26101,US,"With full ripeness, density and concentration,...",Winemaker's Select,91,8.0,Washington,Columbia Valley (WA),Columbia Valley,Paul Gregutt,@paulgwine,Snoqualmie 2006 Winemaker's Select Riesling (C...,Riesling,Snoqualmie,0.0
10386,US,"A marvelous effort in a difficult year, this t...",NaN,91,9.0,Washington,Columbia Valley (WA),Columbia Valley,Paul Gregutt,@paulgwine,Chateau Ste. Michelle 2011 Riesling (Columbia ...,Riesling,Chateau Ste. Michelle,0.0


etc...


In [110]:
# Find the countries that produce the most wines
data.groupby("country").agg({
    "description": "count",
}).rename(columns={"description": "num_wines"}).sort_values(by="num_wines", ascending=False).head()

,num_wines
country,
US,54504
France,22093
Italy,19540
Spain,6645
Portugal,5691
